In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import nltk
from nltk.util import ngrams

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Train/Chittagong Train Translation.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Test/Chittagong Test Translation.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Validation/Chittagong Validation Translation.csv")

In [5]:
train_data.tail(5)

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
1870,কাউকে মিথ্যা বলে খুশী করার থেকে সত্যি বলে কাঁদ...,kauke mittha bole khushi korar theke sotti bol...,কোনিক্কারে মিথ্যা খইয়েরে কুশী করিবার তুন সত্...,konikkare mittha khoiyere khushi koribar tun...,Chittagong,It's better to be happy for telling a lie than...
1871,"একে অপরের প্রতি যতো বেশী বিশ্বাস থাকবে, তাদের ...","eke oporer proti joto beshi bisshas thakbe, ta...","এইজ্ঞো আরেজ্ঞোর ফ্রতি বোত বিশ্বাস থাইবো, ইতা...","eiggoareggor froti bot bishsas thaibo, itaror ...",Chittagong,"The more trust they have in each other, the he..."
1872,বৃষ্টি বেড়ে তাপমাত্রা কমার সুখবর দিয়েছে আবহা...,brishti bere tapmatra komar sukhobor diyeche a...,বৃষ্টি বাইরলে তাপমাত্রা কমিবার সুখবর দিয়ে আব...,brishti bairle tapmatra komibar sukhobor diye ...,Chittagong,Meteorological office has given the good news ...
1873,পুরো বর্ষা মৌসুমে হাতে গোনা কয়েকদিন মাত্র বৃষ...,puro borsha moushume hate gona koyekdin matro ...,ফুরা বর্ষা মৌসুমোত হাতে গুনা হোয়োজ্ঞা দিন মাত্...,fura borsha mousumot hate guna hoyogga din mat...,Chittagong,After only a handful of days of rain in the en...
1874,খাওয়ার অনেক আগেই যদি ভাত রান্না শেষ হয়ে গিয়ে থ...,khaoyar onek agei jodi vat ranna shesh hoye gi...,খাওয়ার বহুত আগোত যদি ভাত রান্না শেষ ওয় গি তা...,khaoyar bohut agot jodi vat ranna shesh oy gi ...,Chittagong,If the rice is done cooking long before eating...


In [6]:
test_data.head(5)

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
0,তোমার আব্বু কেমন আছে?,tomar abbu kemon ache?,তোয়ার আব্বু কেন আসে?,toyar abbu ken ase?,Chittagong,How is your father?
1,আমার বড় বোনের আজকে মন ভালো নেই,amar boro boner ajke mon valo nei,আর বর বইনোর আজিয়া মন ভালা নাই,ar bor boinor ajiya mon vala nai,Chittagong,My elder sister is not feeling well today
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,tumi ki amake ei kajti kore dite parbe?,তুইকি আরে হাম্মান গরি দিত্তারিবানা?,tuiki are hamman gori dittaribana?,Chittagong,Can you do this for me?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,ei gorome bahire ber hote iccha kore na,এই গরমত বাইর ওইতো মনত নো সার,ei goromot bair oito monot no sar,Chittagong,Don't want to go out in this summer
4,তোমার সব কথাই ভুল,tomar sob kothai vul,তোয়ার বেক হোতাই ভুল,toyar bek hotai vul,Chittagong,Everything you say is wrong


In [7]:
test_data.tail(5)

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
370,তুমি কি আমাকে এক গ্লাস পানি এনে দিতে পারবে ?,tumi ki amake ek glass pani ene dite parbe ?,তুই কি আরে এক্কা্কান গ্লাস ফানি আনি দিত্তারিবা...,tui ki are ekkakan glass fani ani dittariba na?,Chittagong,Can you get me a glass of water?
371,আমি পারবো না পানি দিতে,ami parbo na pani dite,আই ন ফাইজ্জুম ফানি দিতাম,ai n faijjum fani ditam,Chittagong,I can't give water
372,তোমার ভাই পড়ালেখাতে অনেক মেধাবি,tomar vai poralekhate onek medhabi,তোয়ার ভাই ফন্নাফরনত বহুত মেধাবি,toyar vai fonnaforonot bohut medhabi,Chittagong,Your brother is very talented in studies
373,আচ্ছা বলো দেখি বাংলাদেশে কয়টি জেলা?,accha bolo dekhi bangladeshe koyti jela?,আচ্ছা হো সাই বাংলাদেশর জেলা হইয়ান?,accha ho sai bangladeshor jela hoiyan?,Chittagong,Well tell me how many districts in Bangladesh?
374,সামনের দিকে যেয়ে মেয়েটি অনেক হাসবে,samner dike jeye meyeti onek hashbe,সামনর মিক্কা যাইয়েরে মাইয়াফোয়া ইবা বহুত হাসিব দে,samnor mikka jaiyere maiyafoya iba bohut hasib de,Chittagong,The girl will smile a lot going forward


In [8]:
validation_data.head(5)

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
0,বাংলাদেশে ৬৪ টা জেলা,bangladeshe ৬৪ ta jela,বাংলাদেশত ৬৪ ইয়ান জেলা,bangladeshot ৬৪ iyan jela,Chittagong,64 districts in Bangladesh
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,amra sobai gotokal bahire giyechilam,আরা বেয়াক্কুন গতহালিয়া বাইরে গেইলাম,ara beyakkun gotohaliya baire geilam,Chittagong,We all went out yesterday
2,তোমার কথা বলার ধরন অনেক সুন্দর,tomar kotha bolar dhoron onek sundor,তোইয়ার হতা বলার ধরণ বহুত সুন্দর,toiyar hota bolar dhoron bohut sundor,Chittagong,Your way of speaking is very nice
3,বরিশালের মানুষ কেমন হয়?,barishaler manush kemon hoy?,বরিশালর মানুষ হইল্লে অয় দে?,barishalor manush hoille oy de?,Chittagong,How are the people of Barisal?
4,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,খুলনা জেলা কি বহুত সুন্দর নাকি?,khulna jela ki bohut sundor naki?,Chittagong,Khulna district is very beautiful?


In [9]:
validation_data.tail(5)

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
245,তুমি যাও এখান থেকে,tumi jao ekhan theke,তুই যো গয় এরে তুন,tui jo goy ere tun,Chittagong,you go from here
246,আপনি যান এখান থেকে,apni jan ekhan theke,ওরে যন এরে তুন,ore jon ere tun,Chittagong,You go from here
247,তুই যা এখান থেকে,tui ja ekhan theke,তুই যা গয় এরের তুন,tui ja goy erer tun,Chittagong,you go from here
248,আপনারা পার্কে ঘুরে বেড়ান,apnara parke ghure beran,ওনোরা ফার্কত গুরি বেড়াতোক,onora farkot guri beratok,Chittagong,You go for a walk in the park
249,উনি চেষ্টা করেন,uni cheshta koren,ঐইবা চেষ্টা খজ্জিল,oiiba cheshta khojjil,Chittagong,He tries


In [10]:
!pip install transformers torch pandas

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [11]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.2 MB/s eta 0:00:00


In [12]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8f83d9941b81cd01a87828adef1d2d43a10f11ae58dc20f9aaa6698e6e968c69
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [13]:
!pip install sentencepiece

In [14]:
!pip install transformers[sentencepiece]

In [15]:
!transformers-cli cache clear

2024-05-23 19:05:48.549976: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 19:05:48.550028: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 19:05:48.600295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-23 19:05:48.612109: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 19:05:50.208212: W tensorflow/comp

In [16]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00


In [17]:
!pip install accelerate -U

In [18]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-644lhnh3
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-644lhnh3
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=89a20f5541ad7afcfa58f71e7496c1389d6aee7b43c511467273d7449d3a9f52
  Stored in directory: /tmp/pip-ephem-wheel-cache-v620o0lh/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=7bd83a781157c678dad5e04edf9cadc9e00dec5

In [19]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
#!pip install transformers==4.10.3

In [20]:
!pip install torch transformers

In [ ]:
#!pip install accelerate==0.20.3

In [21]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from normalizer import normalize
import pandas as pd
import numpy as np
import nltk
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
train_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Train/Chittagong Train Translation.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Test/Chittagong Test Translation.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Validation/Chittagong Validation Translation.csv")

In [23]:
train_data.head()

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
0,কেমন আছো ?,kemon acho?,কেন আচো?,Ken acho?,Chittagong,How are you?
1,আজকে আমার মন ভালো নেই,ajke amr mon valo nei,আযিয়া আর মন বালা নাই?,ajiya ar mon bala nai?,Chittagong,I'm not feeling well today
2,তুমি কি করো ?,tumi ki koro?,তুঁই কি গরো?,tui ki goro?,Chittagong,what are you doing?
3,এই গরমে আমার কিছু ভালো লাগে না,ei gorome amar kichu valo lage na,এই গরমত আত্তুন কিচু বালা ন লাগের,ei goromot attun kichu bala no lager,Chittagong,I don't like anything this summer
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,cheleti sada ronger ekti shirt pore eshechilo,ফোয়াইবা সাদা রংওর উজ্ঞা শার্ট ফরি আইস্যিল,foyaiba sada rongor ugga shirt fori aissilo,Chittagong,The boy came wearing a white shirt


In [24]:
train_data = train_data.drop(columns=['banglish_speech ','chittagong_banglish_speech ','region_name ','english_speech'],axis=1)
train_data.head()

,bangla_speech,chittagong_bangla_speech
0,কেমন আছো ?,কেন আচো?
1,আজকে আমার মন ভালো নেই,আযিয়া আর মন বালা নাই?
2,তুমি কি করো ?,তুঁই কি গরো?
3,এই গরমে আমার কিছু ভালো লাগে না,এই গরমত আত্তুন কিচু বালা ন লাগের
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,ফোয়াইবা সাদা রংওর উজ্ঞা শার্ট ফরি আইস্যিল


In [25]:
test_data = test_data.drop(columns=['banglish_speech ','chittagong_banglish_speech ','region_name ','english_speech'],axis=1)
test_data.head()

,bangla_speech,chittagong_bangla_speech
0,তোমার আব্বু কেমন আছে?,তোয়ার আব্বু কেন আসে?
1,আমার বড় বোনের আজকে মন ভালো নেই,আর বর বইনোর আজিয়া মন ভালা নাই
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,তুইকি আরে হাম্মান গরি দিত্তারিবানা?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,এই গরমত বাইর ওইতো মনত নো সার
4,তোমার সব কথাই ভুল,তোয়ার বেক হোতাই ভুল


In [26]:
train_data.rename(columns={'bangla_speech ': 'input_text', 'chittagong_bangla_speech '	: 'labels'}, inplace=True)
train_data.head()

,input_text,labels
0,কেমন আছো ?,কেন আচো?
1,আজকে আমার মন ভালো নেই,আযিয়া আর মন বালা নাই?
2,তুমি কি করো ?,তুঁই কি গরো?
3,এই গরমে আমার কিছু ভালো লাগে না,এই গরমত আত্তুন কিচু বালা ন লাগের
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,ফোয়াইবা সাদা রংওর উজ্ঞা শার্ট ফরি আইস্যিল


In [27]:
# Rename the columns to match the expected format
test_data.rename(columns={'bangla_speech ': 'input_text', 'chittagong_bangla_speech '	: 'labels'}, inplace=True)
test_data.head()

,input_text,labels
0,তোমার আব্বু কেমন আছে?,তোয়ার আব্বু কেন আসে?
1,আমার বড় বোনের আজকে মন ভালো নেই,আর বর বইনোর আজিয়া মন ভালা নাই
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,তুইকি আরে হাম্মান গরি দিত্তারিবানা?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,এই গরমত বাইর ওইতো মনত নো সার
4,তোমার সব কথাই ভুল,তোয়ার বেক হোতাই ভুল


In [28]:
validation_data.head()

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
0,বাংলাদেশে ৬৪ টা জেলা,bangladeshe ৬৪ ta jela,বাংলাদেশত ৬৪ ইয়ান জেলা,bangladeshot ৬৪ iyan jela,Chittagong,64 districts in Bangladesh
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,amra sobai gotokal bahire giyechilam,আরা বেয়াক্কুন গতহালিয়া বাইরে গেইলাম,ara beyakkun gotohaliya baire geilam,Chittagong,We all went out yesterday
2,তোমার কথা বলার ধরন অনেক সুন্দর,tomar kotha bolar dhoron onek sundor,তোইয়ার হতা বলার ধরণ বহুত সুন্দর,toiyar hota bolar dhoron bohut sundor,Chittagong,Your way of speaking is very nice
3,বরিশালের মানুষ কেমন হয়?,barishaler manush kemon hoy?,বরিশালর মানুষ হইল্লে অয় দে?,barishalor manush hoille oy de?,Chittagong,How are the people of Barisal?
4,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,খুলনা জেলা কি বহুত সুন্দর নাকি?,khulna jela ki bohut sundor naki?,Chittagong,Khulna district is very beautiful?


In [29]:
validation_data = validation_data.drop(columns=['banglish_speech ','chittagong_banglish_speech ','region_name ','english_speech'],axis=1)
validation_data.head()

,bangla_speech,chittagong_bangla_speech
0,বাংলাদেশে ৬৪ টা জেলা,বাংলাদেশত ৬৪ ইয়ান জেলা
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,আরা বেয়াক্কুন গতহালিয়া বাইরে গেইলাম
2,তোমার কথা বলার ধরন অনেক সুন্দর,তোইয়ার হতা বলার ধরণ বহুত সুন্দর
3,বরিশালের মানুষ কেমন হয়?,বরিশালর মানুষ হইল্লে অয় দে?
4,খুলনা জেলা কি অনেক সুন্দর?,খুলনা জেলা কি বহুত সুন্দর নাকি?


In [30]:
# Rename the columns to match the expected format
validation_data.rename(columns={'bangla_speech ': 'input_text', 'chittagong_bangla_speech '	: 'labels'}, inplace=True)
validation_data.head()

,input_text,labels
0,বাংলাদেশে ৬৪ টা জেলা,বাংলাদেশত ৬৪ ইয়ান জেলা
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,আরা বেয়াক্কুন গতহালিয়া বাইরে গেইলাম
2,তোমার কথা বলার ধরন অনেক সুন্দর,তোইয়ার হতা বলার ধরণ বহুত সুন্দর
3,বরিশালের মানুষ কেমন হয়?,বরিশালর মানুষ হইল্লে অয় দে?
4,খুলনা জেলা কি অনেক সুন্দর?,খুলনা জেলা কি বহুত সুন্দর নাকি?


In [31]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_name = "facebook/mbart-large-50"
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="ro_RO")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
'''import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer
#https://huggingface.co/docs/transformers/model_doc/mt5
model_name = "google/mt5-small" # The variations it has -> mt5-small: 6, mt5-base: 12,mt5-large: 24, mt5-xl: 24, mt5-xxl: 24
model = MT5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the saved model
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Movies/Chittagong_model.pt")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Movies/Chittagong_tokenizer.json")

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [32]:
import torch
from normalizer import normalize
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os

In [ ]:
""""model_name = "csebuetnlp/banglat5"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [33]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
'''
# Save the model
model.save_pretrained("/content/drive/MyDrive/movie/sylhet_model.pt")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/movie/sylhet_tokenizer.json")'''

'\n# Save the model\nmodel.save_pretrained("/content/drive/MyDrive/movie/sylhet_model.pt")\n\n# Save the tokenizer\ntokenizer.save_pretrained("/content/drive/MyDrive/movie/sylhet_tokenizer.json")'

In [34]:
from torch.utils.data import Dataset, DataLoader

In [35]:
class Seq2SeqDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        """
        Args:
            data: A DataFrame containing 'input_text' and 'labels' columns.
            tokenizer: A Hugging Face tokenizer.
            max_length: Maximum sequence length.
        """
        self.input_text = data['input_text'].apply(normalize).tolist()
        self.labels = data['labels'].apply(normalize).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self, idx):
        input_text = self.input_text[idx]
        label_text = self.labels[idx]

        # Tokenize the input text
        input_encodings = self.tokenizer(
            input_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Tokenize the label text to get its 'input_ids' and 'attention_mask'
        label_encodings = self.tokenizer(
            label_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encodings['input_ids'].squeeze(),
            'attention_mask': input_encodings['attention_mask'].squeeze(),
            'labels': label_encodings['input_ids'].squeeze(),
        }


In [36]:
# Create train , test and validation datasets
train_dataset = Seq2SeqDataset(train_data, tokenizer)
test_dataset = Seq2SeqDataset(test_data, tokenizer)
validation_dataset = Seq2SeqDataset(validation_data, tokenizer)

# Create train , test and validation dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)  #batch_size=32
test_dataloader = DataLoader(test_dataset, batch_size=16) #batch_size=32
validation_dataloader = DataLoader(validation_dataset, batch_size=16) #batch_size=32


# **For training dataset**

In [ ]:
# Assuming you have already created the train_dataloader
for data_point in train_dataloader:
    input_ids = data_point['input_ids']
    attention_mask = data_point['attention_mask']
    labels = data_point['labels']
''' print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)
'''

' print("Input IDs:", input_ids)\n    print("Attention Mask:", attention_mask)\n    print("Labels:", labels)\n'

# **For test dataset**

In [ ]:
# Assuming you have already created the test_dataloader
for data_point in test_dataloader:
    input_ids = data_point['input_ids']
    attention_mask = data_point['attention_mask']
    labels = data_point['labels']
''' print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)
'''

' print("Input IDs:", input_ids)\n    print("Attention Mask:", attention_mask)\n    print("Labels:", labels)\n'

# **For validation dataset**

In [ ]:
# Assuming you have already created the validation_dataloader
for data_point in validation_dataloader:
    input_ids = data_point['input_ids']
    attention_mask = data_point['attention_mask']
    labels = data_point['labels']
'''    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)'''

'    print("Input IDs:", input_ids)\n    print("Attention Mask:", attention_mask)\n    print("Labels:", labels)'

# **from training dataset , I have done only one example decoding**

In [ ]:
# Assuming you have already created the train_dataloader with shuffle
for data_point in train_dataloader:
    input_ids = data_point['input_ids'][0]  # Assuming you want the first example
    attention_mask = data_point['attention_mask'][0]
    labels = data_point['labels'][0]

    # Decode input_ids and labels
    input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    output_text = tokenizer.decode(labels, skip_special_tokens=True)

    print("Input Text:", input_text)
    print("Output Text:", output_text)

    # Exit the loop after processing the first example
    break


Input Text: আমার নৌকায় ঘুরতে অনেক ভালো লাগে
Output Text: আঁর নৌকাত গুইরতো বউত ভালা লাগে


# **from testing dataset , I have done only one example decoding**

In [ ]:
# Assuming you have already created the test_dataloader
for data_point in test_dataloader:
    input_ids = data_point['input_ids'][0]  # Assuming you want the first example
    attention_mask = data_point['attention_mask'][0]
    labels = data_point['labels'][0]

    # Decode input_ids and labels
    input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    output_text = tokenizer.decode(labels, skip_special_tokens=True)

    print("Input Text:", input_text)
    print("Output Text:", output_text)

    # Exit the loop after processing the first example
    break


Input Text: তোমার আব্বু কেমন আছে?
Output Text: তোয়ার আব্বু কেন আসে?


# **from validation dataset , I have done only one example decoding**

In [ ]:
# Assuming you have already created the validation_dataloader
for data_point in validation_dataloader:
    input_ids = data_point['input_ids'][0]  # Assuming you want the first example
    attention_mask = data_point['attention_mask'][0]
    labels = data_point['labels'][0]

    # Decode input_ids and labels
    input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    output_text = tokenizer.decode(labels, skip_special_tokens=True)

    print("Input Text:", input_text)
    print("Output Text:", output_text)

    # Exit the loop after processing the first example
    break


Input Text: বাংলাদেশে ৬৪ টা জেলা
Output Text: বাংলাদেশত ৬৪ ইয়ান জেলা


In [37]:
# Move the model to the device (CPU or GPU)
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (fi

In [ ]:
#pip install transformers==4.30


In [38]:
from transformers import TrainingArguments
from transformers import AdamW

from torch.optim import AdamW

# Create a custom optimizer using torch.optim.AdamW
custom_optimizer = AdamW(
    model.parameters(),
    lr=1e-3,  # Learning rate
    eps=1e-8,  # Epsilon value to prevent division by zero
    weight_decay=0.01,  # Weight decay (L2 regularization)
)

#if you have 1,000 training examples and a batch size of 100, you would have 10 iterations in each epoch (1,000 / 100 = 10)
'''
This parameter determines how many small batches are accumulated before performing a weight update.
In your code, it's set to 8, which means you'll accumulate gradients over 8 small batches before performing a weight update.
This effectively simulates a larger batch size without requiring more GPU memory.
So, you are updating the model's weights less frequently compared to the number of actual batches processed.
'''



'''
Learning rate determines how quickly the model learns from the data.
The learning rate scheduler type is set to "cosine_with_restarts," which is a type of learning rate schedule.
 Warmup steps are the number of initial training steps with a smaller learning rate, and weight decay introduces L2 regularization to the optimizer.
'''



'''
Number of Iterations per Epoch = Number of Training Samples / Batch Size
Total Iterations = Number of Iterations per Epoch * Number of Epochs
'''
# Define the TrainingArguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Movies',
    num_train_epochs=60,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="epoch",
    save_strategy="no",  # Align save strategy with evaluation strategy
    ##save_total_limit=1,
    #save_steps=700,
    learning_rate=1e-3,
    do_train=True,
    do_eval=True,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
    #load_best_model_at_end=True,
    lr_scheduler_type="cosine_with_restarts",
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/Movies',
    logging_steps=10,  # Log every 10 steps,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [39]:
from transformers import DataCollatorForSeq2Seq

# Create a data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,  # Your Hugging Face tokenizer
    model=model,
    padding=True,
    max_length=128,
    label_pad_token_id=tokenizer.pad_token_id,
)


In [40]:

# Define the Trainer with the custom optimizer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=(custom_optimizer, None),  # Pass the custom optimizer here
)


# **Training start here**

In [ ]:
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
0,0.709800,0.561317
1,0.659800,0.486660


# **Saving model and tokenizer**

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Save the model
model.save_pretrained("/content/drive/MyDrive/Movies/Chittagong_mBart50_model.pt")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Movies/Chittagong_mBart50_tokenizer.json")


# **Loading trained model and tokenizer**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the saved model
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Movies/Chittagong_model.pt")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Movies/Chittagong_tokenizer.json")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
!pip install datasets

In [ ]:
!pip install python-Levenshtein

In [ ]:
!pip install jiwer

In [ ]:
# Move the model to the device (CPU or GPU)
model.to(device)

In [ ]:
!pip install rouge-score
#https://github.com/google-research/google-research/tree/master/rouge
#https://huggingface.co/spaces/evaluate-metric/rouge [Different types of ROUGE scores]

In [ ]:
!pip install evaluate

# **Loading evaluation metrics**

In [ ]:
import torch
import Levenshtein
from evaluate import load
# Define the move_to_device function
def move_to_device(batch, device):
    if isinstance(batch, torch.Tensor):
        return batch.to(device)
    elif isinstance(batch, list):
        return [move_to_device(item, device) for item in batch]
    elif isinstance(batch, dict):
        return {key: move_to_device(value, device) for key, value in batch.items()}
    else:
        return batch  # If it's not a tensor, list, or dict, leave it as is

# Load the evaluation metric for Character Error Rate (CER) and Word Error Rate (WER) and Exact Match(em)
cer_metric = load("cer")
wer_metric = load("wer")
meteor = load('meteor')
exact_match_metric = load("exact_match")

# Load BLEU and ROUGE metrics
bleu_metric = load("bleu")
rouge_metric = load('rouge')

# Initialize lists to store generated translations and references
generated_translations = []
references = []

# Generate translations for the test dataset
for batch in test_dataloader:
    # Move the batch to CUDA
    batch = move_to_device(batch, 'cuda')

    input_text = batch['input_ids']  # Access the input_text using the correct key
    labels = batch['labels']  # Access the labels using the correct key

    # Generate translations
    translation_ids = model.generate(input_text, max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)

    # Move the translation_ids to CPU to decode
    translation_ids = translation_ids.to('cpu')

    generated_translation = tokenizer.batch_decode(translation_ids, skip_special_tokens=True)

    generated_translations.extend(generated_translation)
    references.extend(tokenizer.batch_decode(labels, skip_special_tokens=True))  # Decoding the label IDs

# Make sure to move generated_translations back to CPU for evaluation if necessary
generated_translations = [translation if not isinstance(translation, str) else translation for translation in generated_translations]



In [ ]:
print("Number of generated translations:", len(generated_translations))
print("Number of references:", len(references))

Number of generated translations: 375
Number of references: 375


In [ ]:
print(generated_translations)

['তোঁয়ার আব্বু ইবা আচে নে?', 'আর ভোইনর আযিয়া মন বালা নাই', 'তুঁই কি আঁরে হনারে হাইত্তাম গরিত ফাইরতো গরিব নে?', 'এই গরমত বাইর ইচ্ছা গরিয়েরে নো গরে', 'তোঁয়ার সব হতাই বুল', 'তোঁয়ারমত হারাপ দেইক্কি আঁই আঁর এক্কানও দেহি', 'বিয়ার লাই আঁর লাই আঁর লাই যার', 'আঁরে গতহা ইবার ফোয়ারিত দেহা গইরতো', 'ফোয়াইবা আঁর এক্কান বউত বালা লাগে', 'মাইয়াফোয়া ইবা রংররংর শাড়িত আইস্যিল', 'আর দুইদিন ফরতি বিয়া অইবো', 'ফোয়াই এহন আর বিয়াত ফরিত নো ফারে', 'মাইফোয়াইবা সমত গুমত্তুন উইটত্তুন ফরিত বইসসে', 'আর চাচা বিদেশত্তুন আইসসে গতত্তুন আইসসে', 'আঁর বাসার দারোয়াইন বউত বালা মানুষ', 'আঁর বন্দু গুলুনা গুইরতো গেইয়্যুই', 'আঁর চাইতো বউত বালা লাগে', 'আই বিদেশত গরিয়েরে বঊত ঘুইরতো চাই', 'তুঁই আরে বিরক্ত গরো ন', 'আঁরে বউত অফমান গরে', 'আঁর বেজ্ঞুনর লই গম বইরত বউত গম লাগে', 'আর ছোট ভাই এক্কান ফরালেহা গইরতো চায়', 'আঁর চোডো ভাই বিদ্যালয় অত যাইত চায়', 'আঁর চোড ভাইর বিদ্যালয়অত গরি ফরালেহা গইরতো ন ন লাগে', 'আঁই একদিন বিদ্যালয় অর মাডঅত মাডঅত মাতা গইরগিয়ে', 'অনে কি আরে নইবা', 'আঁর বন্দু আযিয়া আযিয়া ফুটবল কেলা হাইতো'

In [ ]:
print(references)

['তোয়ার আব্বু কেন আসে?', 'আর বর বইনোর আজিয়া মন ভালা নাই', 'তুইকি আরে হাম্মান গরি দিত্তারিবানা?', 'এই গরমত বাইর ওইতো মনত নো সার', 'তোয়ার বেক হোতাই ভুল', 'তোয়ার ডইল্লে মরতফোয়া আই এক্কান ও নো দেহি', 'আর লাই ফোওয়া কুজেরদে', 'গতহালিয়া আব্বু আরে এক্কান ফোয়ার ফোয়ারিত দেহা গরিবেল্লাই হইল', 'ফোয়া ইবা আর এক্কেরে গম ন লাগে', 'মাইয়াফোয়া ইবা গোলাফি কালার শাড়ি ফইজ্জেদে', 'আর দুইদিন ফোর বিয়া অইবোদে', 'মারতফোয়া ইবা এহন আর বিকালত খেলিবেল্লাই ন আইয়ে', 'মাঈয়া ফোয়া ইবা বেইন্না ঘুমত্তুন উডি ফন্না ফইওত্তে বইসসে', 'আর চাচা গতহালিয়া বিদেশেত্তুন বাংলাদেশত আইসসে', 'আর বাসার দারওয়ান বহুত ভালা মানুষ', 'আর বান্ধু খুলনা গুইত্ত গিয়ে', 'আর গুরতো বহুত ভালা লাগে', 'আই বিদেশত জাইয়েরে বহুত ঘুইরতাম চাই', 'তুই আরে বিরক্ত ন গইজ', 'তুই আরে বহুত অফমান গইজ্জ', 'আর সকালত উডিয়েরে বই ফইত্তাম বহুত ভালো লাগে', 'আর ভাই ছোড উওা একদমি ফন্না ফরিবেল্লাই ন চায়', 'আর ভাই ছোড ইবা স্কুলত জাইতো ন চায়', 'আর ভাই ছোড ইবা স্কুলত জাইয়েরে ফন্না ফরিবেল্লাই ন চায়', 'আই একদিন স্কুলর মাডত মাথা গুরাইয়েরে ফরি গেইলাম', 'অনেকি 

In [ ]:
# Calculate Character Error Rate (CER) and Word Error Rate (WER)
results_CER = cer_metric.compute(predictions=generated_translations, references=references)
results_WER = wer_metric.compute(predictions=generated_translations, references=references)

# Calculate Exact Match (EM) and METEOR(M)
results_em = exact_match_metric.compute(predictions=generated_translations, references=references)
results_met = meteor.compute(predictions=generated_translations, references=references)

# Calculate Bilingual Evaluation Understudy (BLEU) and Recall-Oriented Understudy for Gisting Evaluation (ROUGE)
results_bleu = bleu_metric.compute(predictions=generated_translations, references=references)
results_rouge = rouge_metric.compute(predictions=generated_translations, references=references)


# Calculate Levenshtein Distance
levenshtein_distances = [Levenshtein.distance(generated, reference) for generated, reference in zip(generated_translations, references)]


# **Printing every evaluation metrics**

In [ ]:
print(results_CER)
print(results_WER)
print(results_em)
print(results_met)
print(results_bleu)
print(results_rouge)
print(levenshtein_distances)

In [ ]:
total_correct = 0
total_samples = len(references)

for generated, reference in zip(generated_translations, references):
    levenshtein_distance = Levenshtein.distance(generated, reference)
    max_length = max(len(generated), len(reference))
    accuracy = 1 - (levenshtein_distance / max_length)
    if accuracy >= 0.8:  # Adjust the threshold as needed
        total_correct += 1

accuracy = total_correct / total_samples
print("Accuracy:", accuracy)


Accuracy: 0.064


# **Saving input text(Chittagong Language) , labels(normal bangla) and generated translations to csv**

In [ ]:
import pandas as pd

# Create a DataFrame to store translations
translation_df = pd.DataFrame({
    'input_text': test_data['input_text'],  # Assuming 'test_data' contains your test dataset
    'labels': references,
    'translations': generated_translations
})

# Save translations to a CSV file
# Save translations to a CSV file
translation_df.to_excel('/content/sample_data/chittagong_translations_BanglaT5.xlsx', index=False)
